In [1]:
library ('tidyverse')
library ('irr')

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.2     v purrr   0.3.4
v tibble  3.0.3     v dplyr   1.0.1
v tidyr   1.1.1     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: lpSolve



## Download data

In [2]:
path_analysis = "D:\\vm_exchange\\PhD_Data\\HealthAssessment"
#df = read_delim(file.path(path_analysis,"HA_all.csv"), ";", escape_double = FALSE, trim_ws = TRUE)
df = read_delim(file.path(path_analysis,"HA_all_20-10-2020.csv"), ";", escape_double = FALSE, trim_ws = TRUE)
#convert into to correct format
#for (v in c('PassageID', 'BirdID', 'Diarrhea', 'MissingToes')){
#    df[[v]] = as.factor(df[[v]])}
c_fac = c('PassageID', 'BirdID', 'Diarrhea', 'MissingToes')
df[c_fac] = lapply(df[c_fac], factor)
c_cont = c('CombPeck','CombColor','FeatherNeck','FeatherWing','FeatherTail','FeatherBreast',
            'Wounds','RFPododermatitis','RFBumblefoot','RFinjuries','LFPododermatitis','LFBumblefoot','Lfinjuries')
for (v in c_cont){
    df[[v]] = as.double(gsub(",", ".", df[[v]]))}
#order with respect to the birdID to ensure comparing the correct birds together
df %>% arrange(desc(Person))
#ATTENTION: verify you have both the same number fo birds manually in the csv file (cant be bother doing this in R)
str(df) #??

Parsed with column specification:
cols(
  .default = col_character(),
  PassageID = col_double(),
  Diarrhea = col_double(),
  MissingToes = col_double()
)

See spec(...) for full column specifications.



Person,PassageID,BirdID,CombPeck,CombColor,FeatherNeck,FeatherWing,FeatherWingNbr,FeatherTail,FeatherTailNbr,FeatherBreast,Diarrhea,Wounds,MissingToes,RFPododermatitis,RFBumblefoot,RFinjuries,LFPododermatitis,LFBumblefoot,Lfinjuries
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mark,2,12vert,0.40,1.40,0.60,0.10,0,0.25,0,1.60,1,0,0,0.00,0.00,0,0.0,0.0,0.00
Mark,1,12vert,0.50,1.00,0.60,0.00,0,0.15,0,1.60,0,0,0,0.00,0.00,0,0.0,0.0,0.00
Mark,2,15gris,0.60,1.00,0.15,0.20,1,0.85,5,0.70,0,0,0,0.00,0.00,0,0.0,0.0,0.00
Mark,1,15gris,0.70,1.40,0.15,0.00,0,0.85,4,0.65,0,0,0,0.00,0.00,0,0.0,0.0,0.00
Mark,1,25orange,0.80,0.80,1.30,0.20,2,0.60,0,1.00,1,0,0,0.25,0.00,0,0.0,0.0,0.00
Mark,2,25orange,0.90,1.15,1.75,0.25,3,0.70,4,0.75,1,0,0,0.00,0.00,0,0.0,0.0,0.00
Mark,1,38black,1.00,1.00,0.85,0.40,5,1.50,10,0.65,1,0,0,0.00,0.00,0,0.0,0.0,0.00
Mark,2,38black,1.00,0.75,0.70,0.30,4,1.70,10,0.80,0,0,0,0.00,0.00,0,0.0,0.0,0.00
Mark,2,44orange,0.85,0.40,0.15,0.10,0,0.20,0,0.60,1,0,0,2.00,0.25,0,1.3,0.0,0.00


tibble [64 x 20] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ Person          : chr [1:64] "Camille" "Camille" "Mark" "Mark" ...
 $ PassageID       : Factor w/ 2 levels "1","2": 2 1 2 1 1 2 2 1 2 1 ...
 $ BirdID          : Factor w/ 16 levels "12vert","15gris",..: 1 1 1 1 2 2 2 2 3 3 ...
 $ CombPeck        : num [1:64] 0.25 0.7 0.4 0.5 1 1 0.6 0.7 0.4 1 ...
 $ CombColor       : num [1:64] 1.5 1.1 1.4 1 1.5 1.1 1 1.4 1 1 ...
 $ FeatherNeck     : num [1:64] 0.3 0.5 0.6 0.6 0.15 0.3 0.15 0.15 1.75 2 ...
 $ FeatherWing     : num [1:64] 0 0 0.1 0 0 0 0.2 0 0.1 0.2 ...
 $ FeatherWingNbr  : chr [1:64] "0" "0" "0" "0" ...
 $ FeatherTail     : num [1:64] 0.25 0.2 0.25 0.15 1 0.8 0.85 0.85 1 0.9 ...
 $ FeatherTailNbr  : chr [1:64] "5-2casse" "3" "0" "0" ...
 $ FeatherBreast   : num [1:64] 1 1.6 1.6 1.6 1.5 1.5 0.7 0.65 1 1.3 ...
 $ Diarrhea        : Factor w/ 2 levels "0","1": 1 1 2 1 1 1 1 1 2 1 ...
 $ Wounds          : num [1:64] 0 0 0 0 0 0 0 0 0 0 ...
 $ MissingToes     : Factor w/ 2 levels "0

In [3]:
df %>% arrange(desc(BirdID))
df

Person,PassageID,BirdID,CombPeck,CombColor,FeatherNeck,FeatherWing,FeatherWingNbr,FeatherTail,FeatherTailNbr,FeatherBreast,Diarrhea,Wounds,MissingToes,RFPododermatitis,RFBumblefoot,RFinjuries,LFPododermatitis,LFBumblefoot,Lfinjuries
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Camille,2,yellow,0.80,0.75,0.20,0.00,0,0.15,"1,5",0.80,0,0.00,0,0.20,0,0.0,0.0,0.0,0.30
Camille,1,yellow,1.00,0.60,0.10,0.10,2,0.15,1,0.50,0,0.00,0,0.75,0,0.0,0.0,0.0,0.40
Mark,2,yellow,0.75,0.80,0.20,0.10,0,0.20,1,0.20,0,0.00,0,0.30,0,0.0,0.0,0.0,0.00
Mark,1,yellow,0.85,0.90,0.65,0.15,0,0.20,1,0.35,0,0.00,0,0.50,0,0.0,0.0,0.0,0.00
Camille,1,white,0.25,1.40,1.60,0.10,1,0.25,"3,5",NA,1,0.00,1,0.00,0,0.0,0.3,0.0,0.00
Camille,2,white,0.50,1.40,1.20,0.30,1/3 bout,0.35,"3,5",2.00,1,0.00,1,0.00,0,0.0,0.0,0.0,0.00
Mark,1,white,0.10,0.90,1.85,0.20,4,0.30,2,2.00,1,0.00,1,0.00,0,0.0,1.8,0.1,0.25
Mark,2,white,0.10,1.00,1.50,0.25,4,0.50,3,1.25,1,0.00,1,0.00,0,0.0,1.5,0.0,0.00
Camille,1,turquoise,0.15,1.40,0.25,0.10,2,0.50,"2,5",1.40,1,0.00,0,0.00,0,0.0,0.0,0.0,0.00


Person,PassageID,BirdID,CombPeck,CombColor,FeatherNeck,FeatherWing,FeatherWingNbr,FeatherTail,FeatherTailNbr,FeatherBreast,Diarrhea,Wounds,MissingToes,RFPododermatitis,RFBumblefoot,RFinjuries,LFPododermatitis,LFBumblefoot,Lfinjuries
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Camille,2,12vert,0.25,1.50,0.30,0.00,0,0.25,5-2casse,1.00,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0
Camille,1,12vert,0.70,1.10,0.50,0.00,0,0.20,3,1.60,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0
Mark,2,12vert,0.40,1.40,0.60,0.10,0,0.25,0,1.60,1,0,0,0.00,0.00,0.00,0.0,0.0,0.0
Mark,1,12vert,0.50,1.00,0.60,0.00,0,0.15,0,1.60,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0
Camille,1,15gris,1.00,1.50,0.15,0.00,0,1.00,40,1.50,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0
Camille,2,15gris,1.00,1.10,0.30,0.00,0,0.80,1/3-1/2,1.50,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0
Mark,2,15gris,0.60,1.00,0.15,0.20,1,0.85,5,0.70,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0
Mark,1,15gris,0.70,1.40,0.15,0.00,0,0.85,4,0.65,0,0,0,0.00,0.00,0.00,0.0,0.0,0.0
Camille,2,25orange,0.40,1.00,1.75,0.10,"0,5",1.00,6,1.00,1,0,0,0.00,0.00,0.00,0.0,0.0,0.1


## Intra observer reliability

In [12]:
#create a file to register data
sink(file.path(path_analysis,'RealiabilityIntra.txt'))

In [13]:
#continuous data
for (person in unique(df[['Person']])){
    if (is.na(person)==FALSE){
        cat(paste0('--------------------------------------------------\n', person),
            file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
        for (var in c_cont){
            cat(paste0('-----------------\n',var), file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
            df_ = data.frame(data.frame(subset(df, (df$PassageID =="1") & (df$Person==person))[[var]]),
                             data.frame(subset(df, (df$PassageID =="2") & (df$Person==person))[[var]]))
            capture.output(icc(df_, model="twoway", type="agreement"), 
                           file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
        }}}

In [14]:
#categorical/binary data
for (person in unique(df[['Person']])){
    if (is.na(person)==FALSE){
        cat(paste0('--------------------------------------------------\n', person),
            file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
        for (var in c('Diarrhea', 'MissingToes')){
            cat(paste0('-----------------\n',var),file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
            df_ = data.frame(data.frame(subset(df, (df$PassageID =="1") & (df$Person==person))[[var]]),
                             data.frame(subset(df, (df$PassageID =="2") & (df$Person==person))[[var]]))
            capture.output(kappa2(df_[,c(1,2)], "unweighted"), 
                           file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
        }}}

## between Mark &cam - Inter observer reliability

In [15]:
#create a file to register data
sink(file.path(path_analysis,'RealiabilityInter.txt'))

In [16]:
#continuous data
for (passage in unique(df[['PassageID']])){
    if (is.na(person)==FALSE){
        cat(paste0('--------------------------------------------------\n', passage),
            file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
        for (var in c_cont){
            cat(paste0('-----------------\n',var), file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
            df_ = data.frame(data.frame(subset(df, (df$PassageID==passage) & (df$Person=="Mark"))[[var]]),
                             data.frame(subset(df, (df$PassageID==passage) & (df$Person=="Camille"))[[var]]))
            capture.output(icc(df_, model="twoway", type="agreement"), 
                           file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
        }}}

In [17]:
#categorical/binary data
for (passage in unique(df[['PassageID']])){
    if (is.na(person)==FALSE){
        cat(paste0('--------------------------------------------------\n', passage),
            file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
        for (var in c('Diarrhea', 'MissingToes')){
            cat(paste0('-----------------\n',var),file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
            df_ = data.frame(data.frame(subset(df, (df$PassageID==passage) & (df$Person=="Mark"))[[var]]),
                             data.frame(subset(df, (df$PassageID==passage) & (df$Person=="Camille"))[[var]]))
            capture.output(kappa2(df_[,c(1,2)], "unweighted"), 
                           file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
        }}}